In [1]:
import os
import pathlib
import yaml
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import (
    WatsonxLLM,
)
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

from utils import utils
from dotenv import load_dotenv

def load_config():
    with open('config.yaml', 'r') as file:
        config = yaml.safe_load(file)
    return config

config = load_config()

load_dotenv()

True

In [2]:
project_id = os.environ["PROJECT_ID"]
model_id = config["watsonx_model"]["llama_2_70b_chat"]
#model_id = ModelTypes.LLAMA_2_70B_CHAT
parameters = {
    GenParams.MAX_NEW_TOKENS: 20,
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.DECODING_METHOD:"sample",
    GenParams.STOP_SEQUENCES:[],
    GenParams.REPETITION_PENALTY: 1,
    GenParams.TEMPERATURE: 0.5
}

credentials = {
    "url": os.environ["GENAI_API"],
    "apikey": os.environ["GENAI_KEY"],
}  

In [3]:
def watsonx_llm():
    model = Model(
        model_id=model_id,
        params=parameters,
        credentials=credentials,
        project_id=project_id,
        )
    llm = WatsonxLLM(model=model)
    return llm


llm = watsonx_llm()

# Prompt to generate response

In [4]:
from faker import Faker
fake = Faker()

# Generate customer names
num_customers = 20  # You can change this to the number of customers you want
customer_names = []

customer_names = [{'first_name': fake.first_name(),
                    'last_name': fake.last_name(),
                    'gender': fake.random_element(elements=('male', 'female'))
                    }
                for _ in range(num_customers)]


print(customer_names)

[{'first_name': 'Ryan', 'last_name': 'Martin', 'gender': 'female'}, {'first_name': 'Alexandra', 'last_name': 'Trujillo', 'gender': 'male'}, {'first_name': 'Jessica', 'last_name': 'Riley', 'gender': 'male'}, {'first_name': 'Brandi', 'last_name': 'Watkins', 'gender': 'female'}, {'first_name': 'Maria', 'last_name': 'Barry', 'gender': 'male'}, {'first_name': 'Brent', 'last_name': 'Scott', 'gender': 'female'}, {'first_name': 'Steven', 'last_name': 'Moran', 'gender': 'female'}, {'first_name': 'Sheila', 'last_name': 'Ramirez', 'gender': 'male'}, {'first_name': 'Daniel', 'last_name': 'Olson', 'gender': 'male'}, {'first_name': 'Richard', 'last_name': 'Harper', 'gender': 'male'}, {'first_name': 'Donna', 'last_name': 'Davis', 'gender': 'female'}, {'first_name': 'Cynthia', 'last_name': 'Brown', 'gender': 'female'}, {'first_name': 'Sandra', 'last_name': 'Zamora', 'gender': 'male'}, {'first_name': 'Brian', 'last_name': 'Kennedy', 'gender': 'female'}, {'first_name': 'Bridget', 'last_name': 'Anderson'

### Reading scenarios from Excel file

In [5]:
df = pd.read_excel("data\demo-scenarios.xlsx")
df["Exclude"] = df["Exclude"].fillna("")
rand = np.random.RandomState(12345)
cust_names = [item for item in rand.choice(customer_names, len(df))]
# print(cust_names)
df["Customer Name"] = [f"{name['first_name'].capitalize()} {name['last_name'].capitalize()}" for name in cust_names]
df["Email"] = [f"{name['first_name'].lower()}.{name['last_name'].lower()}@gmail.com" for name in cust_names]
#df[["Customer Name","Email"]].head()

In [6]:

print(df[['Customer Name','Email']])
print(f"\ncolumns:{df.columns.to_list()}")
print(f"\nshape:{df.shape}")

        Customer Name                         Email
0       Jessica Riley       jessica.riley@gmail.com
1         Brent Scott         brent.scott@gmail.com
2  Alexandra Trujillo  alexandra.trujillo@gmail.com
3         Maria Barry         maria.barry@gmail.com
4      Richard Harper      richard.harper@gmail.com
5         Brent Scott         brent.scott@gmail.com
6       Jessica Riley       jessica.riley@gmail.com
7  Alexandra Trujillo  alexandra.trujillo@gmail.com

columns:['SR No.', 'Agent Name', 'Promotion Claim Code', 'Tracking Number', 'Customer Query 1', 'Content', 'Detail', 'Exclude', 'Response', 'Generated', 'Customer Name', 'Email']

shape:(8, 12)


### Sentiment Analysis Starts Here

In [7]:
sa_prompt = """
[INST] perform sentiment analysis on the email content.[/INST]
[INST] Only provide your response as:
- 'Extremely Negative'
- 'Negative'
- 'Neutral'
- 'Positive'
- 'Extremely Positive'
[/INST]
[INST] Only provide sentiment from given list and do not write anything else in your response.[/INST]

[INST]
{input_email}
[/INST]
"""

print(sa_prompt)


[INST] perform sentiment analysis on the email content.[/INST]
[INST] Only provide your response as:
- 'Extremely Negative'
- 'Negative'
- 'Neutral'
- 'Positive'
- 'Extremely Positive'
[/INST]
[INST] Only provide sentiment from given list and do not write anything else in your response.[/INST]

[INST]
{input_email}
[/INST]



In [8]:
from langchain.prompts import PromptTemplate

def build_prompt_all_queries(email,fixed_template):    
    prompt= PromptTemplate(input_variables=["input_email"],template=fixed_template)
    model_prompt = prompt.format(input_email=email)
    return model_prompt

def sentiment_analysis(customer_query_1):   
    print(f"Sentiment Analysis")     
    prompt= build_prompt_all_queries(customer_query_1,sa_prompt)
    #print(f"model prompt:{prompt}")
    sa_response = llm(prompt)
    return sa_response

df['customer_sentiment'] = df['Customer Query 1'].apply(sentiment_analysis)

Sentiment Analysis
Sentiment Analysis
Sentiment Analysis
Sentiment Analysis
Sentiment Analysis
Sentiment Analysis
Sentiment Analysis
Sentiment Analysis


In [9]:
df['customer_sentiment'] = df['customer_sentiment'].apply(lambda x: x.strip())
# assigning sentiment score on the scale of -2 to +2
sentiment_score = {'Extremely Negative': -2,
                   'Negative': -1,
                   'Neutral': 0,
                   'Positive': 1,
                   'Extremely Positive': 2}
df['sentiment_score'] = df['customer_sentiment'].map(sentiment_score)

In [10]:
df[['customer_sentiment','sentiment_score']]

,customer_sentiment,sentiment_score
0,Neutral,0.0
1,Extremely Negative,-2.0
2,Extremely Negative,-2.0
3,Negative,-1.0
4,Extremely Positive,2.0
5,Negative,-1.0
6,Sentiment: Neutral,NaN
7,Sentiment: Neutral,NaN


In [112]:
PROMPT_TEMPLATE = """
Email is below.
---------------------
{{email}}
---------------------

Based on the email provided, remove salutation, sign-off and only leave the body of the email.

Output:
"""

def extract_email(email):
    return llm(PROMPT_TEMPLATE.replace("{{email}}", email)).strip()

df["Email Extract"] = df["Customer Query 1"].apply(extract_email)

df.to_excel("output/demo-scenarios-processed_v3.xlsx", index=False)

# for idx, row in df.iterrows():
#     response = llm(PROMPT_TEMPLATE.replace("{{email}}", row["Email"])).strip()
#     print(response)

In [113]:
SIGN_OFF = """

Regards,
{{AGENT_NAME}} | Promotions Team
Samsung Electronics Australia
1300 362 603
"""

PROMPT_TEMPLATE = """
Email is below.
---------------------
{{email}}
---------------------

You are an empathetic and friendly customer service agent. 

Reply to a customer based on the provided email.

Include the following in your response,
{{content}}

Do not mention the following in your response,
{{exclude}}

Remember:
Start your response with Dear Sir / Madam,.
End your response with Best regards.

Response:
"""


def reply_email(row):
    content = row["Content"].replace("[DETAIL]", row["Detail"])
    prompt = (
        PROMPT_TEMPLATE.replace("{{email}}", row["Email Extract"])
        .replace("{{content}}", content)
        .replace("{{exclude}}", row["Exclude"])
    )
    response = llm(prompt)
    return response[: response.find("Best regards,")].strip() + SIGN_OFF


df["Generated"] = df.apply(reply_email, axis=1)

df.to_excel("output/demo-scenarios-processed_v3.xlsx", index=False)

# for idx, row in df.iterrows():
#     response = reply_email(row)
#     print("Email:")
#     print(row["Email"])
#     print()
#     print("Response:")
#     print(response)
#     print()
#     print("-" * 100)
#     print()

In [ ]:
df[["SR No.", "Customer Name", "Email", "Customer Query 1", "Generated","customer_sentiment","sentiment_score"]]

In [115]:
df[["SR No.", "Customer Name", "Email", "Customer Query 1", "Generated","customer_sentiment","sentiment_score"]].to_json("output/email_response.json", orient="records")

In [117]:
import json
with open("output/email_response.json") as f:
    email_response = json.load(f)

In [ ]:
email_response